In [6]:
from PIL import Image
import pytesseract
##导入通用包
import numpy as np
import pandas as pd
import os
import json
import re
import base64
import xlwings as xw
##导入腾讯AI api
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.ocr.v20181119 import ocr_client, models

In [2]:
pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13633 sha256=3ee591223df0204e18149ad2cd5f8aca99f008fa8dc53ea31b04bedab7781fa4
  Stored in directory: c:\users\13670\appdata\local\pip\cache\wheels\f1\2f\a5\574c57fb22cfcf24f315c8feda132fd0463a9b07ef78394d07
Successfully built pytesseract
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tencentcloud-sdk-python

Note: you may need to restart the kernel to use updated packages.


In [10]:
#定义函数
def excelFromPictures(picture,SecretId,SecretKey):
    try:
        with open(picture,"rb") as f:
                img_data = f.read()
        img_base64 = base64.b64encode(img_data)
        cred = credential.Credential(SecretId, SecretKey)  #ID和Secret从腾讯云申请
        httpProfile = HttpProfile()
        httpProfile.endpoint = "ocr.tencentcloudapi.com"
 
        clientProfile = ClientProfile()
        clientProfile.httpProfile = httpProfile
        client = ocr_client.OcrClient(cred, "ap-shanghai", clientProfile)
 
        req = models.TableOCRRequest()
        params = '{"ImageBase64":"' + str(img_base64, 'utf-8') + '"}'
        req.from_json_string(params)
        resp = client.TableOCR(req)
        #     print(resp.to_json_string())
 
    except TencentCloudSDKException as err:
        print(err)
 
    ##提取识别出的数据，并且生成json
    result1 = json.loads(resp.to_json_string())
 
    rowIndex = []
    colIndex = []
    content = []
 
    for item in result1['TextDetections']:
        rowIndex.append(item['RowTl'])
        colIndex.append(item['ColTl'])
        content.append(item['Text'])
 
    ##导出Excel
    ##ExcelWriter方案
    rowIndex = pd.Series(rowIndex)
    colIndex = pd.Series(colIndex)
 
    index = rowIndex.unique()
    index.sort()
 
    columns = colIndex.unique()
    columns.sort()
 
    data = pd.DataFrame(index = index, columns = columns)
    for i in range(len(rowIndex)):
        data.loc[rowIndex[i],colIndex[i]] = re.sub(" ","",content[i])
 
    writer = pd.ExcelWriter("../tables/" + re.match(".*\.",f.name).group() + "xlsx", engine='xlsxwriter')
    data.to_excel(writer,sheet_name = 'Sheet1', index=False,header = False)
    writer.save()
 
    #xlwings方案  
    # wb = xw.Book()
    # sht = wb.sheets('Sheet1')
    # for i in range(len(rowIndex)):
    #     sht[rowIndex[i],colIndex[i]].value = re.sub(" ",'',content[i])
    # wb.save("../tables/" + re.match(".*\.",f.name).group() + "xlsx")
    # wb.close()
    

 

pictures = os.listdir(r"C:\Users\13670\Desktop\3")
for pic in pictures:
    excelFromPictures(pic,"AKIDLyOcdSeZjcMtTotoa8PozyiDUVTAkKo2","70ynNK0rTow73jkUO3Rq55psgfNo1W0E")
    print("已经完成" + pic + "的提取.")

FileNotFoundError: [Errno 2] No such file or directory: '4.jpg'

In [ ]:
if not ('tables') in os.listdir():
    os.mkdir("./tables/")
 
os.chdir("./pictures/")
pictures = os.listdir()
for pic in pictures:
    excelFromPictures(pic,"AKIDKRJwJWBhhcXDfreUv618eu3SGrEMFCYj","rbCXhbKtGn57tDI7k8fwxjHZbNyMUP7i")
    print("已经完成" + pic + "的提取.")